<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Config" data-toc-modified-id="Config-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Config</a></span><ul class="toc-item"><li><span><a href="#import" data-toc-modified-id="import-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>import</a></span></li><li><span><a href="#Path" data-toc-modified-id="Path-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Path</a></span></li><li><span><a href="#sys.path" data-toc-modified-id="sys.path-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>sys.path</a></span></li><li><span><a href="#import-from-local" data-toc-modified-id="import-from-local-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>import from local</a></span></li><li><span><a href="#video" data-toc-modified-id="video-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>video</a></span></li></ul></li><li><span><a href="#Main" data-toc-modified-id="Main-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Main</a></span><ul class="toc-item"><li><span><a href="#Definition-of-the-parameters" data-toc-modified-id="Definition-of-the-parameters-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Definition of the parameters</a></span></li><li><span><a href="#Get-Video" data-toc-modified-id="Get-Video-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get Video</a></span></li><li><span><a href="#Load-YOLO" data-toc-modified-id="Load-YOLO-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Load YOLO</a></span></li><li><span><a href="#Log-Preprocess" data-toc-modified-id="Log-Preprocess-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Log Preprocess</a></span></li><li><span><a href="#while-loop" data-toc-modified-id="while-loop-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>while loop</a></span></li></ul></li><li><span><a href="#END" data-toc-modified-id="END-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>END</a></span></li></ul></div>

## Config

### import

In [39]:
import os
import sys
import pathlib
import re
import pprint
import time

import numpy as np
import cv2
print("cv2.__version__ : {}".format(cv2.__version__))

cv2.__version__ : 3.4.4


In [21]:
import timeit
import warnings
from PIL import Image

In [22]:
# tensrflow gpu
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="6,7"

### Path

In [23]:
ROOT_Path = pathlib.Path(os.getcwd()).parents[0]
assert ROOT_Path.exists(), "Not exists. : {}".format(ROOT_Path)
print(ROOT_Path)

/home/sugisaki/workdir/git/deep_sort_yolov3


In [24]:
data_Path = pathlib.Path(
    "/home/sugisaki/workdir/git/ghelia_mizuho-detect-fraud/data/original/video-camera01")
assert data_Path.exists()
print(data_Path)

/home/sugisaki/workdir/git/ghelia_mizuho-detect-fraud/data/original/video-camera01


### sys.path

In [25]:
sys.path.append(str(ROOT_Path))

### import from local

In [26]:
import yolo

from deep_sort import preprocessing
from deep_sort import nn_matching
from deep_sort import detection
from deep_sort import tracker

from tools import generate_detections

### video

In [27]:
# `.`で始まるファイルを無視
# ignore `.` initial filenames
video_filename_list = [
    name
    for name in os.listdir(path=str(data_Path))
    if re.search(pattern=r'^(?!.*^\.).*',
                 string=name,
                 flags=re.IGNORECASE) is not None
]
video_filename_list = sorted(video_filename_list)
pprint.pprint(video_filename_list)

['01_01_1.avi',
 '01_02_1.avi',
 '01_38_1_1.AVI',
 '01_38_1_2.AVI',
 '01_43_1_1.AVI',
 '01_43_1_2.AVI',
 '01_44_1_1.AVI',
 '01_44_1_2.AVI',
 '01_45_1_1.AVI',
 '01_45_1_2.AVI',
 '01_46_0_1.AVI',
 '01_46_0_2.AVI']


## Main

In [28]:
warnings.filterwarnings('ignore')

In [29]:
filepath = str(data_Path / video_filename_list[0])
assert os.path.exists(filepath)

### Definition of the parameters

In [30]:
# Definition of the parameters
max_cosine_distance = 0.3
nn_budget = None
nms_max_overlap = 1.0

In [31]:
# deep_sort 
model_filename = str(ROOT_Path / "model_data" / "mars-small128.pb")
encoder = \
    generate_detections.create_box_encoder(
        model_filename=model_filename,
        input_name    ='images',
        output_name   ='features',
        batch_size    =1)

In [32]:
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker_instance = tracker.Tracker(metric)

### Get Video

In [33]:
video_capture = cv2.VideoCapture(filepath)
frame_total_num = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
print("frame_total_num : {}".format(frame_total_num))

frame_total_num : 239416


In [34]:
writeVideo_flag = True
if writeVideo_flag:
# Define the codec and create VideoWriter object
    w = int(video_capture.get(3))
    h = int(video_capture.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter('output.avi', fourcc, 15, (w, h))
    list_file = open('detection.txt', 'w')
    frame_index = -1 

In [35]:
fps = 0.0

### Load YOLO

In [36]:
yolov3 = yolo.YOLO(model_path   = str(ROOT_Path / "model_data" / "yolo.h5"),
                   anchors_path = str(ROOT_Path / "model_data" / "yolo_anchors.txt"),
                   classes_path = str(ROOT_Path / "model_data" / "coco_classes.txt"))

/home/sugisaki/workdir/git/deep_sort_yolov3/model_data/yolo.h5 model, anchors, and classes loaded.


### Log Preprocess

In [37]:
log_split_len = 1000
if frame_total_num % log_split_len == 0:
    log_interval = frame_total_num // log_split_len
else:
    log_interval = (frame_total_num + log_split_len) // log_split_len

log_idxes = [i for i in range(log_interval-1, frame_total_num, log_interval)]

print("len(log_idxes) : {}".format(len(log_idxes)))

len(log_idxes) : 997


### while loop

In [41]:
#while True:
pct_progress = 0.
start_time = time.time()

for frame_counter in range(frame_total_num):
    ret, frame = video_capture.read()  # frame shape 640*480*3
    if ret != True:
        print("Failed video_capture.read()")
        break
    t1 = timeit.time.time()

    image = Image.fromarray(frame)
    boxs = yolov3.detect_image(image)
    # print("box_num",len(boxs))
    features = encoder(frame,boxs)
    # score to 1.0 here).
    detections = [
        detection.Detection(tlwh=bbox, confidence=1.0, feature=feature)
        for bbox, feature in zip(boxs, features)
    ]
    # Run non-maxima suppression.
    boxes = np.array([d.tlwh for d in detections])
    scores = np.array([d.confidence for d in detections])
    indices = preprocessing.non_max_suppression(boxes, nms_max_overlap, scores)
    detections = [detections[i] for i in indices]

    # Call the tracker
    tracker_instance.predict()
    tracker_instance.update(detections)
    for track in tracker_instance.tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue 
        bbox = track.to_tlbr()
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,255,255), 2)
        cv2.putText(frame, str(track.track_id),(int(bbox[0]), int(bbox[1])),0, 5e-3 * 200, (0,255,0),2)

    for det in detections:
        bbox = det.to_tlbr()
        cv2.rectangle(frame,(int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,0,0), 2)

    if writeVideo_flag:
        # save a frame
        out.write(frame)
        frame_index = frame_index + 1
        list_file.write(str(frame_index)+' ')
        if len(boxs) != 0:
            for i in range(0,len(boxs)):
                list_file.write(str(boxs[i][0]) + ' '+str(boxs[i][1]) + ' '+str(boxs[i][2]) + ' '+str(boxs[i][3]) + ' ')
        list_file.write('\n')

    fps = (fps + (1./(timeit.time.time()-t1))) / 2

    if frame_counter == log_idxes[0]:
        elapsed_time = time.time() - start_time
        pct_progress += 1
        log_idxes.pop(0)
        print(
            "=== {:6.3}% === | frame {:8}/{:8}| fps : {:8.4} | elapsed time : {:8.2}"
            .format(pct_progress, frame_counter, frame_total_num, fps,
                    elapsed_time))

# END Log
pct_progress += 1
log_idxes.pop(0)
print(
    "=== {:6.3}% === | frame {:8}/{:8}| fps : {:8.4} | elapsed time : {:8.2}"
    .format(pct_progress, frame_counter, frame_total_num, fps,
            elapsed_time))
video_capture.release()
if writeVideo_flag:
    out.release()
    list_file.close()
cv2.destroyAllWindows()

===    1.0% === | frame      719/  239416| fps :    14.54 | elapsed time :  5.2e+01
===    2.0% === | frame      959/  239416| fps :    14.28 | elapsed time :  6.9e+01
===    3.0% === | frame     1199/  239416| fps :    14.85 | elapsed time :  8.6e+01
===    4.0% === | frame     1439/  239416| fps :    13.83 | elapsed time :    1e+02
===    5.0% === | frame     1679/  239416| fps :    15.16 | elapsed time :  1.2e+02
===    6.0% === | frame     1919/  239416| fps :    14.29 | elapsed time :  1.4e+02
===    7.0% === | frame     2159/  239416| fps :    15.43 | elapsed time :  1.5e+02
===    8.0% === | frame     2399/  239416| fps :    14.71 | elapsed time :  1.7e+02
===    9.0% === | frame     2639/  239416| fps :    14.15 | elapsed time :  1.9e+02
===   10.0% === | frame     2879/  239416| fps :    14.53 | elapsed time :  2.1e+02
===   11.0% === | frame     3119/  239416| fps :    14.78 | elapsed time :  2.2e+02
===   12.0% === | frame     3359/  239416| fps :    14.98 | elapsed time :  

ValueError: Format specifier missing precision

## END